In [85]:
import csv
import geopandas as gp
import pandas as pd
import  tabula

In [70]:
#Process presidential votes
STATE = 'KY'
DEMOCRAT = 'trump'
REPUBLICAN = 'clinton'
president = {}
reader = csv.reader(open('./election/2016-precinct-president.csv', 'r'))
for row in reader:
    if row[4] != STATE:
        continue
        
    #democr, republican
    id = row[8] + row[13][:4]     
    
    if id in president.keys():
        votes = president[id]
    else:
        votes = [0, 0]
    if row[15] == DEMOCRAT:
        votes[0] += int(row[21])
    if row[15] == REPUBLICAN:
        votes[1] += int(row[21])
    president[id] = votes
    
print (president)
        
    

{'21001D104': [275, 103], '21001C104': [456, 88], '21001A105': [288, 30], '21001C102': [341, 89], '21001A102': [215, 10], '21001AB': [436, 97], '21001F101': [585, 130], '21001A104': [359, 28], '21001D101': [328, 44], '21001B102': [422, 148], '21001E101': [420, 40], '21001F103': [349, 63], '21001D103': [306, 44], '21001G101': [443, 95], '21001G102': [554, 113], '21001B104': [387, 49], '21001E104': [473, 152], '21003C101': [576, 144], '21003C102': [710, 136], '21003A101': [411, 150], '21003A102': [513, 85], '21003AB': [345, 112], '21003D102': [325, 57], '21003E102': [510, 104], '21003D101': [739, 152], '21003E103': [330, 39], '21003B101': [545, 104], '21003B102': [468, 67], '21003E101': [235, 41], '21003B103': [230, 49], '21003D103': [529, 109], '21005C101': [550, 211], '21005C102': [588, 211], '21005A101': [435, 223], '21005A102': [568, 153], '21005AB': [362, 165], '21005F101': [430, 115], '21005F104': [519, 158], '21005D102': [744, 238], '21005D101': [601, 129], '21005E103': [524, 135]

In [71]:
file = "./KY/KY.geojson" 
df = gp.read_file(file)
df.head()

,area,id,type,centerX,centerY,canonical,neighbors,geometry
0,0.002012,21179D105,ORIGINAL,-85.47519600000001,37.849999,KY Nelson D105,21179D103,"MULTIPOLYGON (((-85.47258 37.82863, -85.48071 ..."
1,0.005112,21149C102,ORIGINAL,-87.10202099999999,37.532737,KY McLean C102,None,"MULTIPOLYGON (((-87.15110 37.49865, -87.15485 ..."
2,0.020443,21147F106,ORIGINAL,-84.6542855,36.6619115,KY McCreary F106,None,"MULTIPOLYGON (((-84.60974 36.72119, -84.60625 ..."
3,0.006973,21147A104,ORIGINAL,-84.4995435,36.6219995,KY McCreary A104,None,"MULTIPOLYGON (((-84.60519 36.59878, -84.60444 ..."
4,0.002376,21147F109,ORIGINAL,-84.51780650000001,36.655203,KY McCreary F109,None,"MULTIPOLYGON (((-84.50946 36.62620, -84.50964 ..."


In [72]:
president_votes = df.copy()
president_votes = president_votes.drop(columns = ['area', 'type', 'centerX', 'centerY', 'canonical', 'neighbors', 'geometry'])

president_votes.insert(1, 'democrat', 0)
president_votes.insert(2, 'republican', 0)

president_votes.head()

,id,democrat,republican
0,21179D105,0,0
1,21149C102,0,0
2,21147F106,0,0
3,21147A104,0,0
4,21147F109,0,0


In [78]:
notFound = []
for i, row in president_votes.iterrows():
    id = row['id']
    if id in president.keys():
        president_votes.at[i, 'democrat'] = president[id][0]
        president_votes.at[i, 'republican'] = president[id][1]
        del president[id]
    else:
        notFound.append(id)

In [89]:
print(len(notFound))
print(president_votes[president_votes['republican'] == 0].shape)
print(president)

527
(534, 3)
{'21001AB': [436, 97], '21003AB': [345, 112], '21005AB': [362, 165], '21007AB': [115, 27], '21009AB': [725, 305], '21009A104': [156, 36], '21011A105': [404, 170], '21011AB': [221, 112], '21011B104': [347, 96], '21013AB': [521, 149], '21013B108': [148, 25], '21015B134': [596, 183], '21015A123': [710, 268], '21015AB': [3291, 1424], '21015A122': [481, 139], '21017AB': [296, 231], '21019C110': [3, 0], '21019C111': [84, 42], '21019A122': [8, 1], '21019C112': [306, 118], '21019AB4': [918, 526], '21019AB5': [233, 80], '21021AB': [485, 296], '21023AB': [208, 104], '21025ABSE': [204, 114], '21025Earl': [4, 0], '21027Abse': [462, 156], '21027Huds': [329, 56], '21027Garf': [538, 133], '21027Hard': [884, 364], '21027Clov': [677, 226], '21027McQu': [388, 87], '21027Unio': [432, 131], '21027Irvi': [701, 286], '21027Bewl': [298, 48], '21027Cust': [374, 82], '21027Harn': [284, 78], '21027McDa': [703, 220], '21027Glen': [196, 40], '21027West': [218, 53], '21029A113': [541, 150], '21029C113

In [61]:
pd.DataFrame(president_votes).to_json('./election/KY_president2016.json')

In [62]:
#PROCESS CONGRESSIONAL VOTES 2016
#SENATE
STATE = 'KY'
DEMOCRATIC = 'gray'
REPUBLICAN = 'paul'
senate2016 = {}

reader = csv.reader(open('./election/2016-precinct-senate.csv', 'r'))
for row in reader:
    if row[4] != STATE:
        continue
    #democr, republican
    id = row[8] + row[13][:4]
    
    if id in senate2016.keys():
        votes = senate2016[id]
    else:
        votes = [0, 0]
        
    if row[15] == DEMOCRATIC:
        votes[0] += int(row[21])
    if row[15] == REPUBLICAN:
        votes[1] += int(row[21])
    senate2016[id] = votes
    
print (senate2016)

{'21001D104': [144, 237], '21001C104': [136, 416], '21001A105': [52, 264], '21001C102': [138, 313], '21001A102': [40, 179], '21001AB': [134, 402], '21001F101': [211, 505], '21001A104': [82, 302], '21001D101': [64, 300], '21001B102': [163, 415], '21001E101': [89, 358], '21001F103': [110, 296], '21001D103': [87, 257], '21001G101': [151, 386], '21001G102': [182, 498], '21001B104': [93, 336], '21001E104': [196, 433], '21003C101': [197, 537], '21003C102': [190, 667], '21003A101': [177, 395], '21003A102': [139, 469], '21003AB': [137, 334], '21003D102': [95, 290], '21003E102': [160, 471], '21003D101': [220, 698], '21003E103': [79, 297], '21003B101': [160, 495], '21003B102': [104, 429], '21003E101': [55, 220], '21003B103': [71, 204], '21003D103': [156, 473], '21005C101': [327, 470], '21005C102': [351, 484], '21005A101': [318, 378], '21005A102': [282, 467], '21005AB': [220, 350], '21005F101': [218, 350], '21005F104': [285, 422], '21005D102': [405, 614], '21005D101': [245, 513], '21005E103': [24

In [65]:
#HOUSE
STATE = 'KY'
DEMOCRAT = ('yarmuth', 'gaskins', 'kemper', 'sidle', 'silde' )
REPUBLICAN = ('comer', 'bratcher', 'corley', 
              'whitfield', 'guthrie', 'massie', 'rogers', 'barr')
house2016 = {}

reader = csv.reader(open('./election/2016-precinct-house.csv', 'r', encoding = "utf-8"))
for row in reader:
    if row[4] != STATE:
        continue
    #democrat, republican
    id = row[8] + row[13][:4]
    
    if id in house2016.keys():
        votes = house2016[id]
    else:
        votes = [0, 0]
        
    if row[15] in DEMOCRAT:
        votes[0] += int(row[21])
    elif row[15] in REPUBLICAN:
        votes[1] += int(row[21])
    house2016[id] = votes
    
print (house2016)

{'21001D104': [179, 537], '21001C104': [193, 867], '21001A105': [90, 516], '21001C102': [167, 690], '21001A102': [36, 388], '21001AB': [204, 859], '21001F101': [232, 1131], '21001A104': [101, 621], '21001D101': [65, 632], '21001B102': [301, 835], '21001E101': [80, 776], '21001F103': [145, 641], '21001D103': [108, 572], '21001G101': [203, 843], '21001G102': [238, 1085], '21001B104': [115, 705], '21001E104': [259, 958], '21003C101': [238, 1177], '21003C102': [229, 1418], '21003A101': [261, 834], '21003A102': [191, 1000], '21003AB': [209, 715], '21003D102': [135, 623], '21003E102': [264, 959], '21003D101': [282, 1497], '21003E103': [96, 641], '21003B101': [192, 1067], '21003B102': [133, 928], '21003E101': [93, 441], '21003B103': [91, 458], '21003D103': [218, 1039], '21005C101': [217, 585], '21005C102': [246, 583], '21005A101': [247, 447], '21005A102': [182, 564], '21005AB': [159, 402], '21005F101': [123, 435], '21005F104': [168, 527], '21005D102': [258, 762], '21005D101': [143, 611], '210

In [66]:
congress_votes = df.copy()
congress_votes = congress_votes.drop(columns = ['area', 'type', 'centerX', 'centerY', 'canonical', 'neighbors', 'geometry'])

congress_votes.insert(1, 'democrat', 0)
congress_votes.insert(2, 'republican', 0)

congress_votes.head()

,id,democrat,republican
0,21179D105,0,0
1,21149C102,0,0
2,21147F106,0,0
3,21147A104,0,0
4,21147F109,0,0


In [67]:
notFound = []
for i, row in congress_votes.iterrows():
    id = row['id']
    
    if id in senate2016.keys():       
        congress_votes.at[i, 'democrat'] = senate2016[id][0]
        congress_votes.at[i, 'republican'] = senate2016[id][1]
    elif id in house2016.keys():
        democrats = congress_votes.at[i, 'democrat']
        congress_votes.at[i, 'democrat'] = democrats + house2016[id][0]
        republicans = congress_votes.at[i, 'republican']
        congress_votes.at[i, 'republican'] = republicans + house2016[id][1]
    else:
        notFound.append(id)

In [68]:
print(congress_votes[congress_votes['democrat'] == 0].shape)
notFound

(531, 3)


['21111B174',
 '21111J131',
 '21115B107',
 '21115B102',
 '21205C105',
 '21085F104',
 '21107A202',
 '21107A201',
 '21107E222',
 '21107C213',
 '21107B210',
 '21093D106',
 '21093G111',
 '21093H106',
 '21093G109',
 '21013A106',
 '21183C105',
 '21037A903',
 '21037A901',
 '21037A902',
 '21037A807',
 '21037A913',
 '21037A909',
 '21099B102',
 '21099A102',
 '21111B167',
 '21179A106',
 '21111B176',
 '21111I103',
 '21111B143',
 '21115C103',
 '21115C106',
 '21115B101',
 '21115A103',
 '21115C109',
 '21107A203',
 '21107G234',
 '21085F102',
 '21107F228',
 '21107G240',
 '21107G236',
 '21027B101',
 '21027A102',
 '21027F101',
 '21027E101',
 '21027C101',
 '21143X001',
 '21129C103',
 '21129D102',
 '21129C101',
 '21027F102',
 '21099C103',
 '21099C101',
 '21099D101',
 '21093H105',
 '21093E103',
 '21093F106',
 '21093G103',
 '21093F105',
 '21093A102',
 '21093A105',
 '21093G110',
 '21093F101',
 '21093G101',
 '21093G104',
 '21101E103',
 '21111F110',
 '21111B164',
 '21111M120',
 '21037B403',
 '21013A107',
 '2111

In [69]:
pd.DataFrame(congress_votes).to_json("./election/KY_congress2016.json")

,id,democrat,republican
0,420010040,NaN,NaN
1,420010010,NaN,NaN
2,420010260,NaN,NaN
3,420010500,NaN,NaN
4,420010100,NaN,NaN


In [82]:
file_path = './election/president_KY.pdf'
print(tabula.read_pdf(file_path , pages='all'))

AttributeError: module 'tabula' has no attribute 'read_pdf'